In [ ]:
import logging
from pathlib import Path

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
)

In [ ]:
mosaic_input = Path("data/DEM_BLANCA.TIF")
mosaic_output = Path("results/DEM_BLANCA_MR.TIF")
spatial_resolution = 10

In [ ]:
with rasterio.open(mosaic_input) as src:
    x_res = y_res = spatial_resolution

    transform, width, height = calculate_default_transform(
        src.crs,
        {"init": "EPSG:32718"},
        src.width,
        src.height,
        *src.bounds,
        resolution=(x_res, y_res),
    )

    kwargs = src.meta.copy()
    kwargs.update(
        {
            "crs": "EPSG:32718",
            "transform": transform,
            "width": width,
            "height": height,
            "nodata": 0,
            "driver": "GTiff"
        }
    )

    with rasterio.open(mosaic_output, "w", **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs="EPSG:32718",
                resampling=Resampling.cubic,
                src_nodata=None,
                dst_nodata=0,
            )

logging.info(f"Resampled DEM saved in {mosaic_output}")